# Dynamic programming for sequence alignment

This notebook implements dynamic programming for sequence alignment, both local and global version (Needleman-Wunsch and Smith-Waterman algorithms). It also outputs several optimal solutions, if there are multiple, but stops at a predetermined threshold.

To use the notebook, set input sequences and other parameters in the cell below, then run all cells. Found alignments are printed in section [Found alignments](#found-alignments). 

## Input and settings

In the cell below you can change the input for the algorithm. Make sure that two sequences are enclosed in qoutes, such as `X="ACGT"`. Typically, the match score should be positive and gap and mismatch scores should be negative.

In [3]:
# X and Y are the two sequences to be aligned
# they may contain only characters A, C, G, and T
X = "CATGTCGTA"
Y = "CAGTCCTAGA"
# below are scores for match, mismatch, and gap
score_match = 1
score_mismatch = -1
score_gap = -1
# whether to perform local or global alignment (value True or False)
local_alignment = True
# how many optimal alignments to print at most
maximum_printed_alignments = 10

# here we check that the inputs are valid
import re
assert re.match("^[ACGT]*$", X), "Invalid characters found in sequence X"
assert re.match("^[ACGT]*$", Y), "Invalid characters found in sequence Y"
assert isinstance(score_match, int), "score_match must be an integer"
assert isinstance(score_mismatch, int), "score_mismatch must be an integer"
assert isinstance(score_gap, int), "score_gap must be an integer"
assert isinstance(local_alignment, bool), "local_alignment must be True or False"
assert isinstance(maximum_printed_alignments, int) and maximum_printed_alignments > 0, \
    "maximum_printed_alignments must be a positive integer"

## Computing tables A and B

Here is the program for computing the main dynamic programming table $A$ with scores and table $B$ with back tracing arrows. Since we want to print multiple solutions, we keep all optimal arrows. Therefore table $B$ has 4 boolean values in each cell, saying if this can be start of the optimal alignment, if the alignment can continue diagonally, up and left. 

In [4]:
# we will use numpy arrays for storing the dynamic programming tables
import numpy as np

# lengths of the two input sequences
n = len(X)
m = len(Y)

# create numpy arrays A and B of size n+1 x m+1
# A is filled with zeros
# B has 4 boolean values in each cell, all filled with False
A = np.zeros((n+1, m+1))
B = np.zeros((n+1, m+1, 4), dtype=bool)

# initialize the first row and column of A and B
B[0][0][0] = True  # alignment can start in upper left corner
for i in range(1, n+1):
    if not local_alignment:
        # for global alignment, initialize first column with gap penalties and up arrows
        A[i][0] = i * score_gap
        B[i][0][2] = True    
    else:
        B[i][0][0] = True  # local alignment can start anywhere

for j in range(1, m+1):
    if not local_alignment:
        # for global alignment, initialize first row with gap penalties and left arrows
        A[0][j] = j * score_gap
        B[0][j][3] = True
    else:
        B[0][j][0] = True  # local alignment can start anywhere

# the main dynamic programming loop
for i in range(1, n+1):
    for j in range(1, m+1):
        # compute score for aligning X[i-1] with Y[j-1]
        # (note that X and Y are 0-based, while A and B are 1-based)      
        if X[i-1] == Y[j-1]:
            aligned_score = score_match
        else:
            aligned_score = score_mismatch

        # for local alignment, we can also start a new alignment here with score 0
        if local_alignment:
            start_score = 0
        else:
            # for global alignment, starting a new alignment here is not possible
            # so we set this option to -infinity
            start_score = -np.inf

        # compute the 4 options for the last column of alignment and take the maximum
        options = np.array([start_score, 
                            A[i-1][j-1]+aligned_score, 
                            A[i-1][j]+score_gap, 
                            A[i][j-1]+score_gap])
        A[i][j] = max(options)
        # set the arrows in B for all options that achieve the maximum
        B[i][j] = (options == A[i][j])


## Tracing back alignments

Below is the code for finding and printing alignments according to arrows in matrix $B$. Note even if we have only one alignment, the code works in quadratic time. Computer scientists, do you see why and could you improve this so that a single alignment is printed in linear time?

In [5]:
def find_ends(A, local_alignment):
    """Find the positions of the ends of optimal alignments in matrix A.
    For local alignment, return all positions with the maximum score.
    For global alignment, return only the bottom-right position."""
    if local_alignment:
        max_val = np.max(A)
        positions = np.argwhere(A == max_val)
        return positions
    else:
        return [(A.shape[0]-1, A.shape[1]-1)]

def alignment_extend(current_alignment, char_x, char_y):
    """Add characters char_x and char_y as a new first column to the current alignment."""
    return [char_x + current_alignment[0], 
            char_y + current_alignment[1],
            current_alignment[2]]

def backtrace(B, pos, local_alignment, X, Y, current_alignment, alignments, max_alignments):
    """Recursively backtrace in matrix B from position pos.
    current_alignment is the end of the alignment built so far.
    alignments is the list of all found alignments.
    max_alignments is the maximum number of alignments to find."""

    # if we have found enough alignments, stop
    if len(alignments) >= max_alignments:
        return

    # look at the arrows in B at position pos
    i, j = pos
    directions = B[i][j]

    if directions[0]: 
        # alignment can start here, so we add the current alignment to the list
        alignments.append(current_alignment + [pos])
       
    if directions[1]:
        # alignment continues with a match/mismatch (diagonally)
        # we call backtrace recursively from the diagonal position
        backtrace(B, (i-1, j-1), local_alignment, X, Y, 
                  alignment_extend(current_alignment, X[i-1], Y[j-1]), 
                  alignments, max_alignments)
        
    if directions[2]:  # similarly for arrow up
        backtrace(B, (i-1, j), local_alignment, X, Y, 
                  alignment_extend(current_alignment, X[i-1], "-"), 
                  alignments, max_alignments)
        
    if directions[3]:  # similarly for arrow left
        backtrace(B, (i, j-1), local_alignment, X, Y, 
                  alignment_extend(current_alignment, "-", Y[j-1]), 
                  alignments, max_alignments)

def print_alignments(alignments, A):
    """Print the found alignments along with their scores and positions.
    Each alignment is a list of 4 elements:
    - aligned sequence X
    - aligned sequence Y
    - position (i,j) in matrix A where the alignment ends
    - position (i,j) in matrix A where the alignment starts"""
    for (idx, aln) in enumerate(alignments):
        print(f"Alignment {idx+1}, total score:", A[aln[2][0]][aln[2][1]])
        print(f"{aln[0]} Positions {aln[3][0]+1} to {aln[2][0]} out of {n}")
        print(f"{aln[1]} Positions {aln[3][1]+1} to {aln[2][1]} out of {m}")
        print()

In [6]:
# find all ends of optimal alignments
ends = find_ends(A, local_alignment)
# use recursive backtrace to find all optimal alignments (or up to the specified maximum)
alignments = []
for end in ends:
    backtrace(B, end, local_alignment, X, Y, ["","", end], 
              alignments, maximum_printed_alignments)

## Found alignments

Below we print input parameters and found alignments.

In [7]:
print("X:", X)
print("Y:", Y)
print("Local alignment:", local_alignment)
print("Maximum number of printed alignments:", maximum_printed_alignments)
print("Scores: match =", score_match, "mismatch =", score_mismatch, "gap =", score_gap)
print()

print_alignments(alignments, A)

X: CATGTCGTA
Y: CAGTCCTAGA
Local alignment: True
Maximum number of printed alignments: 10
Scores: match = 1 mismatch = -1 gap = -1

Alignment 1, total score: 5.0
CATGTCGTA Positions 1 to 9 out of 9
CA-GTCCTA Positions 1 to 8 out of 10



## Matrix A

Below we print full matrix $A$.

In [8]:
print("Table A:")
print(A)

Table A:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 2. 1. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 1. 1. 2. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 1. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 3. 2. 1. 1. 0. 0. 0.]
 [0. 1. 0. 0. 2. 4. 3. 2. 1. 0. 0.]
 [0. 0. 0. 1. 1. 3. 3. 2. 1. 2. 1.]
 [0. 0. 0. 0. 2. 2. 2. 4. 3. 2. 1.]
 [0. 0. 1. 0. 1. 1. 1. 3. 5. 4. 3.]]
